<a href="https://colab.research.google.com/github/Method-for-Software-System-Development/Cloud_Computing/blob/develop/logic/Fault_controller.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Navigate to project folder and ensure we're on the 'develop' branch
import os, subprocess, sys

REPO_DIR = "/content/Cloud_Computing"
if not os.path.isdir(REPO_DIR):
    subprocess.run([
        "git", "clone", "-b", "develop",
        "https://github.com/Method-for-Software-System-Development/Cloud_Computing.git",
        REPO_DIR
    ], check=True)

# Update & checkout develop if repo already exists
subprocess.run(["git", "-C", REPO_DIR, "fetch", "origin"], check=True)
subprocess.run(["git", "-C", REPO_DIR, "checkout", "develop"], check=True)
subprocess.run(["git", "-C", REPO_DIR, "pull"], check=True)

# Install importnb (only needed once per session)
%pip install importnb --quiet

# Add firebase folder to Python path and import its notebook as module
sys.path.append(os.path.join(REPO_DIR, "firebase"))  # for FireBase.ipynb
sys.path.append(REPO_DIR)                            # for FAULT_RULES.ipynb
from importnb import Notebook

with Notebook():
    import FireBase as fb                           # now fb.add_user, fb.get_user, etc. are available
    from FAULT_RULES import FAULT_RULES as fr       # now FAULT_RULES are aviable to use


firebase_url = fb.firebase_url

omer_cohen 050-111-2222 200
shira_gold 050-222-3333 200
daniel_bar 050-333-4444 200
roni_benami 050-444-5555 200
lior_levi 050-555-6666 200
yael_elyashiv 050-666-7777 200
noa_katz 050-777-8888 200
matan_tal 050-888-9999 200
or_peled 050-999-0000 200
tom_segal 050-000-1111 200
Daniel Bar | 050-333-4444 | Robotics Engineer
Lior Levi | 050-555-6666 | Backend Developer
Matan Tal | 050-888-9999 | Mechanical Engineer
Noa Katz | 050-777-8888 | UX Designer
Omer Cohen | 050-111-2222 | Automation Engineer
Or Peled | 050-999-0000 | System Engineer
Roni Ben Ami | 050-444-5555 | QA Engineer
Shira Gold | 050-222-3333 | Electrical Engineer
Tom Segal | 050-000-1111 | Production Engineer
Yael Elyashiv | 050-666-7777 | Hardware Engineer


In [3]:
import requests
from datetime import datetime

def process_faults_from_database():
    active_faults = fb.get_active_faults()
    existing_titles = set()

    if isinstance(active_faults, dict):
        existing_titles = {
            fault["title"]
            for fault in active_faults.values()
            if isinstance(fault, dict) and "title" in fault
        }

    for folder in ["indoor", "outdoor"]:
        url = f"{firebase_url}sensor_readings/{folder}.json"
        response = requests.get(url)
        if response.status_code != 200:
            continue

        data = response.json()
        if not isinstance(data, dict):
            continue

        for reading in data.values():
            if not isinstance(reading, dict):
                continue

            for sensor_type, value in reading.items():
                try:
                    value = float(value)
                except (ValueError, TypeError):
                    continue  # skip "N/A" or malformed values

                key = f"{sensor_type.lower()}_{folder}"
                for rule in fr.get(key, []):
                    min_val = rule.get("min")
                    max_val = rule.get("max")

                    if (min_val is None or value >= min_val) and (max_val is None or value < max_val):
                        if rule["title"] in existing_titles:
                            continue

                        title_key = f"{rule['title']} ({datetime.now().strftime('%d-%m-%Y %H:%M:%S')})"

                        fault = {
                            "sensor": rule["sensor"],
                            "severity": rule["severity"],
                            "title": rule["title"],
                            "value": value,
                            "status": "Active",
                            "repaired_by": "",
                            "timestamp": datetime.now().isoformat(),
                            "actions": rule["actions"]
                        }

                        fb.add_active_fault(title_key, fault)
                        existing_titles.add(rule["title"])
